# Lecture 33: Regression Inference

In [ ]:
from datascience import *
import numpy as np

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline
np.set_printoptions(legacy='1.13')

## Linear regression

In [ ]:
def standard_units(any_numbers):
    "Convert any array of numbers to standard units."
    return (any_numbers - np.mean(any_numbers)) / np.std(any_numbers)  

def correlation(t, x, y):
    """Return the correlation coefficient (r) of two variables."""
    return np.mean(standard_units(t.column(x)) * standard_units(t.column(y)))

def slope(t, x, y):
    """The slope of ther regression line (original units)."""
    r = correlation(t, x, y)
    return r * np.std(t.column(y)) / np.std(t.column(x))

def intercept(t, x, y):
    """The intercept of the regression line (original units)."""
    return np.mean(t.column(y)) - slope(t, x, y) * np.mean(t.column(x))

def fit(table, x, y):
    """Return the height of the regression line at each x value."""
    a = slope(table, x, y)
    b = intercept(table, x, y)
    return a * table.column(x) + b

def predict(table, x, y, new_x):
    """Return the prediction for new_x using linear regression."""
    a = slope(table, x, y)
    b = intercept(table, x, y)
    return a * new_x + b    

def plot_residuals(t, x, y):
    """Plot a scatter diagram and residuals."""
    t.scatter(x, y, fit_line=True)
    actual = t.column(y)
    fitted = fit(t, x, y)
    residuals = actual - fitted
    print('r:   ', correlation(t, x, y))
    print('RMSE:', np.mean(residuals**2)**0.5)
    t.select(x).with_column('Residual', residuals).scatter(0, 1)

## Prediction from a sample, not a population

In [ ]:
# We've been treating this table as a population.  It's not really.
galton = Table.read_table('galton.csv')

heights = Table().with_columns(
    'MidParent', galton.column('midparentHeight'),
    'Child', galton.column('childHeight')
)

heights_pred = heights.with_column(
    'Predicted', fit(heights, 'MidParent', 'Child')
)

plot_residuals(heights_pred, 'MidParent', 'Child')

In [ ]:
# How differently could the prediction have come out?
sample_heights = heights.sample()
sample_heights_pred = sample_heights.with_column(
    'Predicted', fit(sample_heights, 'MidParent', 'Child')
)

plot_residuals(sample_heights_pred, 'MidParent', 'Child')
print('Prediction for MidParent=70 is Child=', predict(sample_heights_pred, 'MidParent', 'Child', 70))

## Bootstrap CI for prediction

In [ ]:
# Bootstrap the predicted height at midparent = 70
predicted_heights = make_array()

for i in np.arange(5000):
    sample_heights = heights.sample()
    pred_height = predict(sample_heights, 'MidParent', 'Child', 70)
    predicted_heights = np.append(predicted_heights, pred_height)

# Compute CI    
left = percentile(2.5, predicted_heights)
right = percentile(97.5, predicted_heights)

# Visualize
Table().with_column('Prediction', predicted_heights).hist(bins=np.arange(66.8, 67.8, 0.1), unit='tenth inch')
plots.xlabel('predicted child height for midparent=70')
plots.plot([left, right], [0, 0], color='yellow', lw=8);
print('Approximate 95%-confidence interval of regression height:')
print('(', left, ',', right, ')')

## Prediction intervals

As of Monday, someone was at 286 gestational days, and will max out at 293.  Can we predict the birth weight for each day, as a 95% CI?

In [ ]:
baby = Table.read_table('baby.csv')
baby.show(3)

In [ ]:
# Does linear regression look appropriate?
plot_residuals(baby, 'Gestational Days', 'Birth Weight')

**Note:** RMSE is about 1 lb!  So our predictions could still be off by a sizable amount.

In [ ]:
# A prediction for x=288
x = 288
a = slope(baby, 'Gestational Days', 'Birth Weight')
b = intercept(baby, 'Gestational Days', 'Birth Weight')
predicted_y = a * x + b
baby.scatter('Gestational Days', 'Birth Weight', fit_line=True)
plots.scatter(x, predicted_y, color='gold', s=200);

In [ ]:
predicted_y

In [ ]:
print(int(predicted_y // 16), 'lbs,', int(round(predicted_y % 16,0)), 'oz')

In [ ]:
# Could those outliers have a big effect?
x = 288

baby_wo_outliers = baby.where('Gestational Days', are.above(200))
a = slope(baby_wo_outliers, 'Gestational Days', 'Birth Weight')
b = intercept(baby_wo_outliers, 'Gestational Days', 'Birth Weight')
r = correlation(baby_wo_outliers, 'Gestational Days', 'Birth Weight')
predicted_y = a * x + b
baby_wo_outliers.scatter('Gestational Days', 'Birth Weight', fit_line=True)
plots.scatter(x, predicted_y, color='gold', s=200);
print('r:', r)

In [ ]:
for i in np.arange(4):
    baby_resample = baby.sample()
    a = slope(baby_resample, 'Gestational Days', 'Birth Weight')
    b = intercept(baby_resample, 'Gestational Days', 'Birth Weight')
    r = correlation(baby_resample, 'Gestational Days', 'Birth Weight')
    predicted_y = a * x + b
    baby_resample.scatter('Gestational Days', 'Birth Weight', fit_line=True)
    plots.scatter(x, predicted_y, color='gold', s=200);
    print('r:', r)

Outliers do have an effect on line and prediction, but absent a good reason (e.g., known errors) we can't exclude them just to get a better fitting line.  

In [ ]:
def bootstrap_prediction(table, x, y, new_xs):
    """Bootstrap a 95% CI for a prediction of each new_x in
    the array new_xs.  Return a table with all the CIs."""

    lefts = make_array()
    rights = make_array()
    
    for new_x in new_xs:
        # Bootstrap resampling
        predictions = make_array()
        for i in np.arange(5000):
            resample = table.sample()
            a = slope(resample, x, y)
            b = intercept(resample, x, y)
            predicted_y = a * new_x + b
            predictions = np.append(predictions, predicted_y)

        # 95% CI
        left = percentile(2.5, predictions)
        right = percentile(97.5, predictions)
        lefts = np.append(lefts, left)
        rights = np.append(rights, right)
        
    intervals = Table().with_columns(
        'new x', new_xs,
        'left', lefts,
        'right', rights
    )
    
    return intervals

In [ ]:
bootstrap_prediction(baby, 'Gestational Days', 'Birth Weight', 
                     make_array(288))

In [ ]:
bootstrap_prediction(baby, 'Gestational Days', 'Birth Weight', 
                     np.arange(286, 286+8))

In [ ]:
# We could report CIs with and without outliers, though, and let
# reader draw their own conclusion
bootstrap_prediction(baby.where('Gestational Days', are.above(200)), 
                     'Gestational Days', 'Birth Weight', np.arange(286, 286+8))

**Q:** Is there a 95% chance that the birth weight of a baby born at 288 gestational days is about 122-125?

A. Yes  
B. No  

<br/><br/><br/><br/><br/>

## A Hypothesis Test

In [ ]:
plot_residuals(baby, 'Maternal Age', 'Birth Weight')

**Q:** Should we attempt to use maternal age to predict birth weight?

A. Yes  
B. No  

<br/><br/><br/><br/><br/>

**Null Hypothesis:** The correlation of Maternal Age to Birth Weight is 0.   
**Alternative Hypothesis:** Not 0.    
**Test statistic:** $|r|$  

How would we simulate under the null hypothesis?  We need to generate data sets whose actual population parameter of $r$ is 0, but might randomly vary a little.  Those data sets need to have same number of rows as our `baby` table.  

We could try to do that by resampling from `baby`, but that would be wrong! We don't actually know what its population $r$ value is.

Luckily, we have the function `r_scatter`, which had code in it for producing data sets with a given correlation.  From it, we'll extract the code we need.

In [ ]:
def r_scatter(r):
    "Generate a scatter plot with a correlation approximately r"
    plots.figure(figsize=(5,5))
    x = np.random.normal(0, 1, 1000)
    z = np.random.normal(0, 1, 1000)
    y = r*x + (np.sqrt(1-r**2))*z
    plots.scatter(x, y)
    plots.xlim(-4, 4)
    plots.ylim(-4, 4)

In [ ]:
r_scatter(0)

In [ ]:
def simulate_r(r, size):
    x = np.random.normal(0, 1, size)
    z = np.random.normal(0, 1, size)
    y = r*x + (np.sqrt(1-r**2))*z
    return Table().with_columns('x', x, 'y', y)

In [ ]:
t = simulate_r(0, 10)
t

In [ ]:
correlation(t, 'x', 'y')

In [ ]:
num_repetitions = 5000

def test_statistic(obs_r, hyp_r):
    return np.abs(obs_r - hyp_r)

def bootstrap_test_correlation(table, x, y, hyp_r):
    """Hypothesis test for whether correlation of x and y is hyp_r."""
    statistics = make_array()
    for i in np.arange(num_repetitions):
        resample = simulate_r(hyp_r, table.num_rows)  # corrected
        resample_r = correlation(resample, 'x', 'y')
        statistics = np.append(statistics, test_statistic(resample_r, hyp_r))

    observed = test_statistic(correlation(table, x, y), hyp_r)

    Table().with_column('Test statistic', statistics).hist()
    plots.scatter(observed, 0, color='red', s=80);
    
    p = np.sum(statistics > observed) / num_repetitions  # corrected
    print('p =', p)

In [ ]:
bootstrap_test_correlation(baby, 'Maternal Age', 'Birth Weight', 0)

**Outcome:** Do not reject null hypothesis.  
**Conclusion:**  Maternal Age not a good variable to use in predicting Birth Weight.

**Follow up question:** What is the correlation, if not 0?

In [ ]:
num_repetitions = 5000

def bootstrap_ci_correlation(table, x, y):
    """95% confidence interval for the correlation of x and y."""
    correlations = make_array()
    for i in np.arange(num_repetitions):
        resample = table.sample()
        resample_r = correlation(resample, x, y)
        correlations = np.append(correlations, resample_r)

    left = percentile(2.5, correlations)
    right = percentile(97.5, correlations)

    Table().with_column('Correlation', correlations).hist()
    plots.plot([left,right], [0,0], color='yellow', lw=8);

    print("95% confidence interval is (", left, ", ", right, ")")

In [ ]:
bootstrap_ci_correlation(baby, 'Maternal Age', 'Birth Weight')

95% CI contains 0.0, so we cannot reject the null hypothesis at a p-value of 0.05.

## Update on our prediction

In [ ]:
# A prediction for x=288
x = 288
r = correlation(baby, 'Gestational Days', 'Birth Weight')
rmse = np.std(baby.column('Birth Weight')) * np.sqrt(1 - r**2)
a = slope(baby, 'Gestational Days', 'Birth Weight')
b = intercept(baby, 'Gestational Days', 'Birth Weight')
predicted_y = a * x + b
baby.scatter('Gestational Days', 'Birth Weight', fit_line=True)
plots.scatter(x, predicted_y, color='gold', s=200);
laurence = 111 # 6 lbs 15 oz
plots.scatter(x, laurence, color='dodgerblue', s=200);
print('We predicted: ', predicted_y)
print('Actual:       ', laurence)
print('RMSE:         ', rmse)